### TODO: Parse wget-script to extract text from XML


In [1]:
INPUT_FILE = r'wget_aftonbladet.sh'
# SET_ONLY_TEXT = True
# deactivated = 0
count = 0
LIMIT = 10
#try:
with open(INPUT_FILE, 'r') as fp:
    for line in fp:
        if line[0] != '#':
            print(line, end="")
        if count == LIMIT:
            break
        count=count+1
                


mkdir -p 1831/ && wget --continue -O "1831/Aftonbladet_1831_01_03_bib4345612_18310103_0_s_0001_alto.xml" https://data.kb.se/dark-38655/bib4345612_18310103_0_s_0001_alto.xml # 187K
mkdir -p 1831/ && wget --continue -O "1831/Aftonbladet_1831_01_03_bib4345612_18310103_0_s_0002_alto.xml" https://data.kb.se/dark-38655/bib4345612_18310103_0_s_0002_alto.xml # 194K
mkdir -p 1831/ && wget --continue -O "1831/Aftonbladet_1831_01_03_bib4345612_18310103_0_s_0003_alto.xml" https://data.kb.se/dark-38655/bib4345612_18310103_0_s_0003_alto.xml # 172K
mkdir -p 1831/ && wget --continue -O "1831/Aftonbladet_1831_01_03_bib4345612_18310103_0_s_0004_alto.xml" https://data.kb.se/dark-38655/bib4345612_18310103_0_s_0004_alto.xml # 197K
mkdir -p 1831/ && wget --continue -O "1831/Aftonbladet_1831_01_03_bib4345612_18310103_0_s.mets.metadata" https://data.kb.se/dark-38655/bib4345612_18310103_0_s.mets.metadata # 34K
mkdir -p 1831/ && wget --continue -O "1831/Aftonbladet_1831_01_03_representation.jsonld" https://data

The following code works to turn XML into plain text OCR. Can be used to loop over data from previous cell and generate text files.

In [ ]:
import codecs
import sys
import xml.etree.ElementTree as ET
from urllib.request import urlopen

altourl = 'https://data.kb.se/dark-38655/bib4345612_18310103_0_s_0001_alto.xml'
altourl = 'https://data.kb.se/dark-32256/bib4345612_18630811_0_s_0003_alto.xml'


if sys.stdout.encoding != 'UTF-8':
    sys.stdout = codecs.getwriter('utf-8')(sys.stdout.buffer, 'strict')

namespace = {'alto-1': 'http://schema.ccs-gmbh.com/ALTO',
             'alto-2': 'http://www.loc.gov/standards/alto/ns-v2#',
             'alto-3': 'http://www.loc.gov/standards/alto/ns-v3#',
             'alto-mkc': 'http://www.loc.gov/standards/alto/alto-v2.0.xsd'
             }
#tree = ET.parse(sys.argv[1])

with urlopen(altourl) as f:

    ## THIS CODE PATCHES THE XML CODE BEFORE PARSING
    corrected_tag = b'<alto xmlns="http://www.loc.gov/standards/alto/ns-v2#" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:schemaLocation="http://www.loc.gov/standards/alto/ns-v2# http://www.loc.gov/standards/alto/alto-v2.0.xsd">\r\n'
    alto_list = f.readlines()
    #print("Line to replace: ", alto_list[1])
    alto_list[1] = corrected_tag
    #print("Corrected line: ", alto_list[1])
    #print("Line to replace: ", alto_list[-1])
    alto_list[-1] = b'</alto>\r\n'
    #print("Corrected line: ", alto_list[-1])
    ## ## ##

    tree = ET.ElementTree(ET.fromstringlist(alto_list))
    xmlns = tree.getroot().tag.split('}')[0].strip('{')
    if xmlns in namespace.values():
        for lines in tree.iterfind('.//{%s}TextLine' % xmlns):
            sys.stdout.write('\n')
            for line in lines.findall('{%s}String' % xmlns):
                text = line.attrib.get('CONTENT') + ' '
                sys.stdout.write(text)
    else:
        print('ERROR: Not a valid ALTO file (namespace declaration missing)')
